# Question Answering Demo

This notebook demonstrates the multi-agent RAG pipeline:
1. Intent classification (Router Agent)
2. Hybrid retrieval (Retriever Agent)
3. Cross-encoder reranking
4. Answer generation with citations (Reasoning Agent)
5. Confidence scoring

## Setup

In [ ]:
import os
import sys
import django

# Add backend to path
sys.path.insert(0, os.path.abspath('..'))
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
django.setup()

print("Django setup complete!")

In [ ]:
from rag.services import RAGOrchestrator
from documents.models import Document, DocumentChunk
from chat.models import ChatSession, ChatMessage

# Check available documents
documents = Document.objects.filter(status='READY')
print(f"Available documents: {documents.count()}")
for doc in documents:
    print(f"  - {doc.title} ({doc.num_chunks} chunks)")

## 1. Initialize RAG Orchestrator

The RAG Orchestrator manages the multi-agent workflow using LangGraph.

In [ ]:
# Initialize the orchestrator
orchestrator = RAGOrchestrator()

print("RAG Orchestrator initialized!")
print(f"Top-K retrieval: {orchestrator.top_k}")
print(f"Graph nodes: {list(orchestrator.graph.nodes.keys()) if hasattr(orchestrator.graph, 'nodes') else 'N/A'}")

## 2. Simple Question Answering

Let's ask a simple question about the documents.

In [ ]:
# Ask a question
query = "What is machine learning?"

result = orchestrator.process_query(query)

print("=" * 60)
print(f"Query: {query}")
print("=" * 60)
print(f"\nIntent: {result['metadata'].get('intent', 'N/A')}")
print(f"\nAnswer:\n{result['answer']}")
print(f"\nConfidence: {result['metadata'].get('confidence_score', 'N/A'):.2%}")

if result['citations']:
    print(f"\nCitations ({len(result['citations'])}):\n")
    for i, citation in enumerate(result['citations']):
        print(f"  [{i+1}] {citation['document_title']}")
        print(f"      Page: {citation.get('page', 'N/A')}")
        print(f"      Snippet: {citation['snippet'][:100]}...")

## 3. Complex Question with Multiple Sources

In [ ]:
# More complex question
query = "What are the different types of machine learning and their applications?"

result = orchestrator.process_query(query)

print("=" * 60)
print(f"Query: {query}")
print("=" * 60)
print(f"\nAnswer:\n{result['answer']}")
print(f"\nConfidence: {result['metadata'].get('confidence_score', 'N/A'):.2%}")
print(f"Chunks retrieved: {result['metadata'].get('num_retrieved', 'N/A')}")

## 4. Question with No Relevant Context

Let's see how the system handles questions outside the document scope.

In [ ]:
# Question outside document scope
query = "What is the weather forecast for tomorrow?"

result = orchestrator.process_query(query)

print("=" * 60)
print(f"Query: {query}")
print("=" * 60)
print(f"\nAnswer:\n{result['answer']}")
print(f"\nConfidence: {result['metadata'].get('confidence_score', 'N/A')}")

## 5. Conversation with History

The system can maintain conversation context.

In [ ]:
# First question
chat_history = []

query1 = "What is supervised learning?"
result1 = orchestrator.process_query(query1, chat_history)

print("Q1:", query1)
print("A1:", result1['answer'][:300] + "...")

# Update history
chat_history.append({"role": "user", "content": query1})
chat_history.append({"role": "assistant", "content": result1['answer']})

In [ ]:
# Follow-up question
query2 = "How does it differ from unsupervised learning?"
result2 = orchestrator.process_query(query2, chat_history)

print("Q2:", query2)
print("A2:", result2['answer'])

## 6. Persian Language Query

The system supports multilingual queries.

In [ ]:
# Persian query
query = "یادگیری ماشین چیست؟"

result = orchestrator.process_query(query)

print("=" * 60)
print(f"Query: {query}")
print("=" * 60)
print(f"\nIntent: {result['metadata'].get('intent', 'N/A')}")
print(f"\nAnswer:\n{result['answer']}")

## 7. Examine Retrieval Details

Let's look at the retrieval process in detail.

In [ ]:
query = "What is deep learning?"

result = orchestrator.process_query(query)

print("Retrieval Statistics:")
print(f"  - Intent: {result['metadata'].get('intent')}")
print(f"  - Initial retrieved: {result['metadata'].get('num_initial_retrieved', 'N/A')}")
print(f"  - After reranking: {result['metadata'].get('num_after_rerank', 'N/A')}")
print(f"  - Reranking applied: {result['metadata'].get('reranking_applied', 'N/A')}")
print(f"  - Average relevance: {result['metadata'].get('avg_relevance', 'N/A')}")
print(f"  - Confidence score: {result['metadata'].get('confidence_score', 'N/A'):.2%}")

print("\nCitation relevance scores:")
for i, citation in enumerate(result['citations']):
    score = citation.get('relevance_score', 'N/A')
    if isinstance(score, float):
        print(f"  [{i+1}] {citation['document_title']}: {score:.4f}")
    else:
        print(f"  [{i+1}] {citation['document_title']}: {score}")

## 8. Store Conversation in Database

Let's save the conversation to the database.

In [ ]:
# Create a chat session
session = ChatSession.objects.create(title="Demo QA Session")

# Store conversation
queries = [
    "What is machine learning?",
    "What are its applications?",
    "How is deep learning different?"
]

chat_history = []

for query in queries:
    # Store user message
    user_msg = ChatMessage.objects.create(
        session=session,
        role=ChatMessage.Role.USER,
        content=query
    )
    
    # Get answer
    result = orchestrator.process_query(query, chat_history)
    
    # Store assistant message
    assistant_msg = ChatMessage.objects.create(
        session=session,
        role=ChatMessage.Role.ASSISTANT,
        content=result['answer'],
        metadata={
            'citations': result['citations'],
            'confidence_score': result['metadata'].get('confidence_score', 0)
        }
    )
    
    # Update history
    chat_history.append({"role": "user", "content": query})
    chat_history.append({"role": "assistant", "content": result['answer']})
    
    print(f"Stored Q&A: {query[:50]}...")

print(f"\nSession ID: {session.id}")
print(f"Total messages: {session.messages.count()}")

In [ ]:
# Retrieve conversation
messages = ChatMessage.objects.filter(session=session).order_by('created_at')

print(f"Conversation in session {session.id}:")
print("=" * 60)

for msg in messages:
    role = "👤 User" if msg.role == 'user' else "🤖 Assistant"
    print(f"\n{role}:")
    print(msg.content[:200] + "..." if len(msg.content) > 200 else msg.content)
    
    if msg.role == 'assistant' and msg.metadata.get('confidence_score'):
        print(f"\n  [Confidence: {msg.metadata['confidence_score']:.2%}]")

## Summary

This notebook demonstrated:

1. **Intent Classification**: Router Agent classifies queries into RAG_QUERY, SUMMARIZE, TRANSLATE, or CHECKLIST
2. **Hybrid Retrieval**: Combines BM25 keyword search with vector similarity search
3. **Cross-Encoder Reranking**: Uses LLM to rerank retrieved chunks for better relevance
4. **Answer Generation**: Reasoning Agent generates grounded answers with citations
5. **Confidence Scoring**: Each answer includes a confidence score based on retrieval quality and LLM confidence
6. **Conversation History**: Maintains context across multiple questions
7. **Multilingual Support**: Works with both English and Persian queries
8. **Database Storage**: Conversations are persisted in PostgreSQL